# Benchmarking FRNN Performance

Explore the techniques described in [this paper](https://reader.elsevier.com/reader/sd/pii/0020019077900709?token=E45C0E1870EA26C21C1F149B6090CE4630A51269D324BE1206B7BF2764FB48B2DDC93F4B86FBFBD8CBDED63B15BBC6DA&originRegion=us-east-1&originCreation=20210428165528).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System imports
import os
import sys
from time import time as tt
import importlib

# External imports
import matplotlib.pyplot as plt
import scipy as sp
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from torch_geometric.data import DataLoader

from itertools import chain
from random import shuffle, sample
from scipy.optimize import root_scalar as root

from torch.nn import Linear
import torch.nn.functional as F
from torch_cluster import knn_graph, radius_graph
import trackml.dataset
import torch_geometric
from itertools import permutations
import itertools
from sklearn import metrics, decomposition
import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.loggers import WandbLogger
from torch.utils.checkpoint import checkpoint

import faiss

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load Model and Dataset

Load the lightning module and setup the model to get the dataset

CHANGE THE EXATRKX PIPELINE LOCATION TO YOUR PARTICULAR LOCATION

In [ ]:
exatrkx_pipeline = '/global/homes/d/danieltm/ExaTrkX/Tracking-ML-Exa.TrkX/Pipelines/TrackML_Example'
sys.path.append(exatrkx_pipeline)

In [3]:
from LightningModules.Embedding.Models.layerless_embedding import LayerlessEmbedding
from LightningModules.Embedding.utils import graph_intersection, build_edges

CHANGE THE CHECKPOINT DIRECTORY TO YOUR PARTICULAR LOCATION

In [4]:
chkpt_dir = "/global/cscratch1/sd/danieltm/ExaTrkX/lightning_checkpoints/CodaEmbeddingStudy/pbn07koj"
chkpt_file = "last.ckpt"
chkpt_path = os.path.join(chkpt_dir, chkpt_file)

In [5]:
model = LayerlessEmbedding.load_from_checkpoint(chkpt_path)

In [6]:
model.hparams["train_split"] = [100,10,10]

In [7]:
model.setup(stage="fit")

In [8]:
model = model.to(device)

# FRNN Testing

## Prepare input data

In [56]:
batch = model.trainset[0].to(device)
with torch.no_grad():
    spatial = model(torch.cat([batch.cell_data, batch.x], axis=-1))

## Run FRNN algorithm

In [60]:
import frnn

In [80]:
r = 0.3

In [81]:
%%time
# first time there is no cached grid
dists, idxs, nn, grid = frnn.frnn_grid_points(
    points1=spatial.unsqueeze(0), points2=spatial.unsqueeze(0), lengths1=None, lengths2=None, K=32, r=r, grid=None, return_nn=False, return_sorted=True
)

ValueError: for now only grid in 2D/3D is supported

In [63]:
%%time
dists, idxs, nn, grid = frnn.frnn_grid_points(
    points1=spatial.unsqueeze(0), points2=spatial.unsqueeze(0), lengths1=None, lengths2=None, K=32, r=r, grid=grid, return_nn=False, return_sorted=True
)

CPU times: user 1.53 ms, sys: 0 ns, total: 1.53 ms
Wall time: 1.09 ms


In [64]:
# Remove the unneccessary batch dimension
idxs = idxs.squeeze()

Convert the array of IDs into an edge list

In [75]:
ind = torch.Tensor.repeat(torch.arange(idxs.shape[0], device=device), (idxs.shape[1], 1), 1).T
positive_idxs = idxs >= 0
edge_list = torch.stack([ind[positive_idxs], idxs[positive_idxs]])

# Remove self-loops
e_spatial = edge_list[:, edge_list[0] != edge_list[1]]

## Accuracy Performance

Find truth

In [76]:
e_bidir = torch.cat([batch.layerless_true_edges, batch.layerless_true_edges.flip(0)], axis=-1) 
e_spatial, y_cluster = graph_intersection(e_spatial, e_bidir, using_weights=False)

In [77]:
t = e_bidir.shape[1]
tp = y_cluster.sum()
p = e_spatial.shape[1]

In [78]:
print(f'Efficiency: {tp / t}, Purity: {tp / p}')

Efficiency: 0.027304455637931824, Purity: 0.0021169965621083975
